# Causal gene regulatory network reconstruction

Cells continuously monitor their internal and external environment and calculate the amount at which each type of protein is needed. This information-processing function is carried out by [gene regulatory networks (GRNs)](https://en.wikipedia.org/wiki/Gene_regulatory_network), which control the rate of production of each protein. Two important classes of regulatory molecules in GRNs are [transcription factors](https://en.wikipedia.org/wiki/Transcription_factor) and [microRNAs](https://en.wikipedia.org/wiki/MicroRNA).

In this tutorial we will use causal inference to reconstruct GRNs from [mRNA](https://en.wikipedia.org/wiki/Messenger_RNA) and [microRNA](https://en.wikipedia.org/wiki/MicroRNA) expression data from the [GEUVADIS study](https://www.nature.com/articles/nature12531). In particular, we will use genetic variants as causal instruments and the [BioFindr software](https://github.com/tmichoel/BioFindr.jl) for model selection, as introduced in the [blessing of dimensionality notebook](2_blessing_of_dimensionality.ipynb).

## Setup the environment

In [1]:
using DrWatson
quickactivate(@__DIR__)

In [23]:
using DataFrames
using Arrow
using CSV
using Gadfly
using Compose
using BioFindr
using Printf
using Graphs

## Load data

This tutorial uses [preprocessed data files](https://github.com/lingfeiwang/findr-data-geuvadis) from the [GEUVADIS study](https://doi.org/10.1038/nature12531). We have mRNA (`t` for transcripts) and microRNA (`m`) expression data:

In [3]:
dt = DataFrame(Arrow.Table(datadir("processed","findr-data-geuvadis", "dt.arrow")));
dm = DataFrame(Arrow.Table(datadir("processed","findr-data-geuvadis", "dm.arrow")));

We also have genotype data for the strongest [eQTLs](https://en.wikipedia.org/wiki/Expression_quantitative_trait_loci) for a subset of mRNAs and microRNAs:

In [4]:
dgt = DataFrame(Arrow.Table(datadir("processed","findr-data-geuvadis", "dgt.arrow")));
dgm = DataFrame(Arrow.Table(datadir("processed","findr-data-geuvadis", "dgm.arrow")));

To construct a GRN, we need a list of transcription factors. We will use a [list](https://humantfs.ccbr.utoronto.ca/allTFs.php) from the [Human Transcription Factors database](https://humantfs.ccbr.utoronto.ca/index.php):

In [5]:
TFs = DataFrame(CSV.File(datadir("processed","findr-data-geuvadis", "TF_names_v_1.01.txt"), header=false))
rename!(TFs, [:"Column1" => :GENE_ID]);

The [preprocessed GEUVADIS data](https://github.com/lingfeiwang/findr-data-geuvadis) has been organized such that each column of the genotype data is the strongest eQTL for the corresponding column in the matching expression data. Usually however, eQTL mapping data will be available in the form of a table with variant IDs, gene IDs, and various eQTL associaion statistics (see the [original GEUVADIS file](https://www.ebi.ac.uk/biostudies/files/E-GEUV-1/E-GEUV-1/analysis_results/EUR373.gene.cis.FDR5.all.rs137.txt.gz) for an example). Let's artificially create such tables for our data (`p` for "pairs"):

In [6]:
dpt = DataFrame(SNP_ID = names(dgt), GENE_ID=names(dt)[1:ncol(dgt)]);
dpm = DataFrame(SNP_ID = names(dgm), GENE_ID=names(dm)[1:ncol(dgm)]);

It is a common observation that TFs don't show strong variation at mRNA level, because their activity is mostly regulated at protein level. As a result, the fraction of TFs with an eQTL is only a bit more than 10%:

In [7]:
[length(TFs.GENE_ID) length(intersect(TFs.GENE_ID, dpt.GENE_ID))]

1×2 Matrix{Int64}:
 1639  179

Because this tutorial is only for illustration purposes, we stick with these TFs as regulators in our network. A biological way to expand the list is to include upstream signalling molecules as "proxies" for TFs, as they often show more variation at mRNA level (see [this paper](https://www.nature.com/articles/ng1165#Sec7) for an example). Or one may simply accept that all networks inferred from omics data are "functional networks" where individual edges may correspond to indirect physical interactions, and use all genes with eQTLs as potential regulators.

## GRN reconstruction with BioFindr

We start by computing the probabilities of causal interactions from the selected TFs to all other genes using the methods explained in the [blessing of dimensionality notebook](2_blessing_of_dimensionality.ipynb). Naturally, we don't have to call the same low-level functions used there. Instead we can call a high-level `findr` function that does everything under the hood. You can read more about this function in the [BioFindr documentation](https://lab.michoel.info/BioFindr.jl/stable/testLLR/) or in the [BioFindr tutorials](https://lab.michoel.info/BioFindrTutorials/). The three first arguments in the call below point to the data and are mandatory (for causal inference). The other arguments are optional: `namesX` tells the function to only use a limited set of variables as candidate regulators (the function itself finds the intersection between `namesX` and the eQTL variables in `dpt`); `FDR` tells the function to determine a cutoff on the edge probabilities such that the overall [FDR](https://en.wikipedia.org/wiki/False_discovery_rate) is (in this case) 20%. Setting an FDR threshold does not reduce computation time, only the size of the ouptut. Hence it is recommended to set the value relatively high and, if necessary, filter for a more stringent threshold later (using the [`globalfdr!`](https://lab.michoel.info/BioFindr.jl/stable/utils/#BioFindr.globalfdr!) function)

In [32]:
dP_TF_mRNA = findr(dt, dgt, dpt; namesX=TFs.GENE_ID, FDR=0.2);

We observe a total of more than 8,000 interactions at the FDR=20% threshold. To see how they are distributed over the individual TFs, we do a bit of [data wrangling](https://dataframes.juliadata.org/stable/). We see that three TFs are responsible for more than half of the interactions:

In [9]:
gdf = groupby(dP_TF_mRNA, :Source);

In [38]:
cdf = sort!(combine(gdf, nrow),:nrow, rev=true);

<div style="background-color:LightYellow; color:black">
<h3>Exercise</h3> 
     Replace the command below to identify causal microRNA -> microRNA interactions and repeat the steps above to find the number of interactions per microRNA.
</div>

In [18]:
dP_miRNA_miRNA = DataFrame(:Source => [], :Target => [], :Probability => [], :qvalue => []);

So far we have reconstructed causal networks *within* a single dataset (mRNA or microRNA expression data) using eQTLs for a subset of variables as causal instruments. We can also reconstruct *bipartite* networks where the source nodes (regulators) come from one dataset and the target nodes from another dataset, as long as the variables in both dataset are measure in the same set of samples. For instance, to idenfity causal interactions *from* TFs *to* microRNAs, we run:

In [13]:
dP_TF_miRNA = findr(dm, dt, dgt, dpt; namesX=TFs.GENE_ID, FDR=0.2);

<div style="background-color:LightYellow; color:black">
<h3>Exercise</h3> 
     Repeat the steps above to find the number of interactions per TF. 
</div>

<div style="background-color:LightYellow; color:black">
<h3>Exercise</h3> 
     Replace the command below to identify causal microRNA -> mRNA interactions and repeat the steps above to find the number of interactions per microRNA.
</div>

dP_miRNA_mRNA = DataFrame(:Source => [], :Target => [], :Probability => [], :qvalue => []);

We can now collect all results in a final dataframe containing our predicted GRN interactions at FDR=20%, sorted by descending probability:

In [21]:
dP = sort!(vcat(dP_TF_mRNA, dP_miRNA_miRNA, dP_TF_miRNA, dP_miRNA_mRNA), :Probability, rev=true);

## Directed acyclic graph reconstruction

In some applications we need graphs that don't contain cycles, so-called [directed acyclic graphs (DAGs)](https://en.wikipedia.org/wiki/Directed_acyclic_graph). For instance, if we want to use the predicted GRN as a structure prior for a [Bayesian network](https://en.wikipedia.org/wiki/Bayesian_network), as done in [this paper](https://doi.org/10.3389/fgene.2019.01196), or if we are looking for a [hierarchical GRN representation](https://doi.org/10.1038/nrg2499). [BioFindr](https://github.com/tmichoel/BioFindr.jl) implements a greedy heuristic to create a DAG from a dataframe of edges, which inserts edges one-by-one into the DAG by descending probability, skipping edges that would introduce a cycle (see the [documentation](https://lab.michoel.info/BioFindr.jl/dev/bayesiannets/#BioFindr.dagfindr!) for more info). We can create a DAG for the complete list of edges:

In [24]:
G_all = dagfindr!(dP);

The DAG information has been added to `dP` in the form of a boolean column indicating whether an edge is included in the DAG or not. Two additional columns, `Source_idx` and `Target_idx` give a mapping from node names (Gene IDs) to numerical IDs; these IDs are used to identify nodes in `G_all`, a [directed graph object](https://juliagraphs.org/Graphs.jl/dev/core_functions/simplegraphs/#Graphs.SimpleGraphs.SimpleDiGraph)  from the [Graphs package](https://juliagraphs.org/Graphs.jl/dev/):

In [34]:
dP;

<div style="background-color:LightYellow; color:black">
<h3>Exercise</h3> 
     Create a [view](https://dataframes.juliadata.org/stable/man/working_with_dataframes/#Taking-a-Subset) of dP showing only the interactions *not* included in the DAG. Do you observe a pattern?
</div>

The regulators (`Source` variables) in our GRN are only a subset of the total set of variables (mRNAs and microRNAs). Since cycles can only occur when the `Target` variable of an edge is also a regulator, it makes more sense to reconstruct DAGs from regulator -> regulator interactions alone. Let's create a new dataframe of these edges:

In [35]:
regulators = unique(dP.Source)
dP_reg = filter(row -> row.Target in regulators, dP[:,1:4]);

and now call `dagfindr!` again:

In [36]:
G_reg = dagfindr!(dP_reg);
dP_reg;

Let's keep only the DAG edges in `dP_reg` and write both `dP` and `dP_reg` to a file for downstream analyses. **Note:** first create the `data/results/findr-data-geuvadis` directory if it does not exist yet!

In [31]:
filter!(row -> row.inDAG_greedy_edges==true, dP_reg);
CSV.write(datadir("results","findr-data-geuvadis", "dP_reg.csv"), dP_reg);
CSV.write(datadir("results","findr-data-geuvadis", "dP.csv"), dP);

"C:\\Users\\tmich\\Teaching\\causal-inference-short-course\\data\\results\\findr-data-geuvadis\\dP.csv"